## Cases Data

In [1]:
import numpy as np
import pandas as pd
import json
import urllib.request

#collect data on the cases based on California counties
website = 'https://data.ca.gov'
url = website + '/api/3/action/datastore_search?resource_id=6a1aaf21-2a2c-466b-8738-222aaceaa168'

cases = pd.DataFrame()
completed = False
total_rows: int
rows_added = 0

#the api that we are requesting data from has a set number of rows per 'page'; 
#will need to parse through all the pages to collect all the data
while completed == False:
    fileobj = urllib.request.urlopen(url)
    response_dict = json.loads(fileobj.read())
    current_data = pd.json_normalize(response_dict['result']['records'])
    cases = cases.append(current_data, ignore_index = True)
    url = website + response_dict['result']['_links']['next']
    total_rows = response_dict['result']['total']
    rows_added += current_data.shape[0]
    if rows_added >= total_rows:
        completed = True

In [2]:
print(cases['area'].unique())

['Alameda' 'Alpine' 'Out of state' 'Amador' 'Butte' 'Calaveras' 'Colusa'
 'California' 'Unknown' 'Contra Costa' 'Glenn' 'Del Norte' 'El Dorado'
 'Fresno' 'Inyo' 'Humboldt' 'Imperial' 'Kern' 'Lake' 'Kings' 'Lassen'
 'Madera' 'Los Angeles' 'Marin' 'Mariposa' 'Mendocino' 'Merced' 'Modoc'
 'Mono' 'Monterey' 'Nevada' 'Napa' 'Orange' 'Placer' 'Plumas' 'Riverside'
 'Sierra' 'Sacramento' 'San Benito' 'San Bernardino' 'San Joaquin'
 'Santa Barbara' 'San Diego' 'San Francisco' 'San Mateo' 'San Luis Obispo'
 'Santa Cruz' 'Shasta' 'Santa Clara' 'Siskiyou' 'Solano' 'Stanislaus'
 'Tehama' 'Sonoma' 'Sutter' 'Trinity' 'Tuolumne' 'Tulare' 'Yolo' 'Ventura'
 'Yuba']


In [3]:
#it looks like there are some unneccesary data in 'County'; 'Unknown' and 'Out of State'
#remove the unneccessary data rows
remove_county_index = cases[cases['area'].isin(['Unknown', 'Out of state'])].index
cases = cases.drop(remove_county_index)

In [4]:
print(cases.dtypes)

cumulative_reported_deaths    object
cumulative_deaths             object
cumulative_positive_tests     object
area                          object
cumulative_cases              object
reported_cases                object
positive_tests                object
cumulative_reported_cases     object
area_type                     object
reported_deaths               object
total_tests                   object
deaths                        object
reported_tests                object
date                          object
cases                         object
_id                            int64
cumulative_total_tests        object
population                    object
dtype: object


In [5]:
#columns are of type 'object', we will need to change their types
numeric_columns = ['cumulative_reported_deaths', 'cumulative_deaths', 'cumulative_positive_tests', 'cumulative_cases',
                   'reported_cases', 'positive_tests', 'cumulative_reported_cases', 'reported_deaths', 'total_tests',
                   'deaths', 'reported_tests', 'cases', 'cumulative_total_tests', 'population']
cases[numeric_columns] = cases[numeric_columns].apply(pd.to_numeric)

string_columns = ['area', 'area_type']
cases[string_columns] = cases[string_columns].astype('string')

#the 'date' of todays entry is left as None, we will add the date in
from datetime import date, timedelta

today = date.today()
cases['date'].fillna(today, inplace = True)
cases['date'] = pd.to_datetime(cases['date'])

#sort values for ease of use in visualization 
cases = cases.sort_values(['area_type', 'area', 'date'], ascending = (False, True, True))

In [6]:
#add columns for cases and deaths based on population
cases['cases_per_100k_pop'] = cases['cases'] / cases['population'] * 100000
cases['cumulative_cases_per_100k_pop'] = cases['cumulative_cases'] / cases['population'] * 100000
cases['deaths_per_100k_pop'] = cases['deaths'] / cases['population'] * 100000
cases['cumulative_deaths_per_100k_pop'] = cases['cumulative_deaths'] / cases['population'] * 100000

## Vaccine Data

In [7]:
#collect vaccination data
url = website + '/api/3/action/datastore_search?resource_id=c020ef6b-2116-4775-b11d-9df2875096ab'

vaccines = pd.DataFrame()
completed = False
total_rows: int
rows_added = 0

while completed == False:
    fileobj = urllib.request.urlopen(url)
    response_dict = json.loads(fileobj.read())
    current_data = pd.json_normalize(response_dict['result']['records'])
    vaccines = vaccines.append(current_data, ignore_index = True)
    url = website + response_dict['result']['_links']['next']
    total_rows = response_dict['result']['total']
    rows_added += current_data.shape[0]
    if rows_added >= total_rows:
        completed = True

In [8]:
print(vaccines['county'].unique())

['All CA Counties' 'Alpine' 'All CA and Non-CA Counties' 'Madera' 'Shasta'
 'Amador' 'Butte' 'Calaveras' 'Colusa' 'Del Norte' 'Glenn' 'Humboldt'
 'Imperial' 'Inyo' 'Kings' 'Lake' 'Lassen' 'Marin' 'Mariposa' 'Mendocino'
 'Merced' 'Modoc' 'Mono' 'Monterey' 'Napa' 'Nevada' 'Plumas' 'San Benito'
 'Santa Barbara' 'Sierra' 'Siskiyou' 'Stanislaus' 'Sutter' 'Tehama'
 'Trinity' 'Tulare' 'Tuolumne' 'Unknown' 'Yuba' 'Outside California'
 'El Dorado' 'Alameda' 'Contra Costa' 'Fresno' 'Kern' 'Los Angeles'
 'Sacramento' 'Orange' 'Placer' 'Riverside' 'San Francisco'
 'San Bernardino' 'San Diego' 'Santa Cruz' 'San Luis Obispo' 'San Joaquin'
 'Santa Clara' 'San Mateo' 'Solano' 'Sonoma' 'Ventura' 'Yolo']


In [9]:
#remove rows for 'Unknown', 'Outside California', and 'All CA and Non-CA Counties'
remove_county_index = vaccines[vaccines['county'].isin(['Unknown', 'Outside California', 'All CA and Non-CA Counties'])].index
vaccines = vaccines.drop(remove_county_index)
#replace 'All CA Counties' with 'California' for consistency
vaccines = vaccines.replace({'county': {'All CA Counties': 'California'}})

In [10]:
vaccines.dtypes

cumulative_pfizer_doses         object
cumulative_total_doses          object
cumulative_fully_vaccinated     object
pfizer_doses                    object
california_flag                 object
moderna_doses                   object
at_least_one_dose               object
county                          object
partially_vaccinated            object
total_doses                     object
fully_vaccinated                object
jj_doses                        object
cumulative_jj_doses             object
cumulative_at_least_one_dose    object
_id                              int64
administered_date               object
total_partially_vaccinated      object
cumulative_moderna_doses        object
dtype: object

In [11]:
#change column types to the correct types
numeric_columns = ['cumulative_pfizer_doses', 'cumulative_total_doses', 'cumulative_fully_vaccinated',
                   'pfizer_doses', 'moderna_doses', 'at_least_one_dose', 'partially_vaccinated', 'total_doses',
                   'fully_vaccinated', 'jj_doses', 'cumulative_jj_doses', 'cumulative_at_least_one_dose',
                   'total_partially_vaccinated', 'cumulative_moderna_doses']
vaccines[numeric_columns] = vaccines[numeric_columns].apply(pd.to_numeric)

string_columns = ['california_flag', 'county']
vaccines[string_columns] = vaccines[string_columns].astype('string')

vaccines['administered_date'] = pd.to_datetime(vaccines['administered_date'])

In [12]:
#sort values based on county and date, but put rows for 'California' first
vaccines = vaccines.sort_values(['county', 'administered_date'], ascending = (True, True))
vaccines_ca = vaccines[vaccines['county'] == 'California']
vaccines_counties = vaccines[vaccines['county'] != 'California']
vaccines = vaccines_ca.append(vaccines_counties)

In [13]:
#merge desired data into one dataframe
cases['county_date'] = cases['area'] + cases['date'].astype(str)
vaccines['county_date'] = vaccines['county'] + vaccines['administered_date'].astype(str)
cases = cases.merge(vaccines[['county_date', 'fully_vaccinated', 'cumulative_fully_vaccinated']], 
                    how = 'left', left_on = 'county_date', right_on = 'county_date')

#vaccination values are null for today, replace with 0
cases[['fully_vaccinated', 'cumulative_fully_vaccinated']] = cases[['fully_vaccinated', 'cumulative_fully_vaccinated']].fillna(0)
#for cumulative vaccination values, use the values from yesterday
today_indexes = cases[cases['date'] == today.isoformat()].index
for i in today_indexes:
    cases.loc[i, 'cumulative_fully_vaccinated'] = cases.loc[i-1, 'cumulative_fully_vaccinated']

#add vaccination values based on proportion
cases['cumulative_vaccination_proportion'] = cases['cumulative_fully_vaccinated'] / cases['population']
cases['fully_vaccinated_per_100k_pop'] = cases['fully_vaccinated'] / cases['population'] * 100000

## Data Visualization

In [14]:
#create methods to easily filter the tables when creating our visualizations
from typing import List

def filter_by_county(county: List[str]) -> pd.DataFrame():
    """return dataframe only with data from 'county'"""
    return cases[cases['area'].isin(county)]

def filter_by_date(date: str) -> pd.DataFrame():
    """return dataframe with data for specific date"""
    return cases[cases['date'] == date]

In [15]:
#we will need a geojson for California counties in order to draw am map for visualization
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/california-counties.geojson') as response:
    counties = json.load(response)

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input
import plotly.express as px

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container([
    #create title
    dbc.Row([
        dbc.Col(html.H1('California County Coronavirus Dashboard',
                        className = 'text-center'),
                width = 12)
    ]),
    
    #create radio selection to build visualizations for 'Cases', 'Vaccine', or 'Death'
    dbc.Row([
        dbc.Col([
            dcc.RadioItems(
                id = 'radio',
                options=[
                    {'label': 'Cases', 'value': 'Cases'},
                    {'label': 'Vaccine', 'value': 'Vaccine'},
                    {'label': 'Death', 'value': 'Death'}
                ],
                value = 'Cases',
                labelStyle = {'padding': '5px', 'margin': 'auto'}
            )  
        ])
    ]),
    
    #create a dropdown menu to select which counties are to be show on the graphs
    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id = 'dropdown',
                options = [
                    {'label': i, 'value': i} for i in cases['area'].unique()
                ],
                value = ['California'],
                multi = True,
                clearable = True,
                placeholder = 'Filter by county ...')
        ], width = 5)
    ], justify = 'center'),
    
    #placeholder to build line graph for daily values
    dbc.Row([
        dbc.Col([
           dcc.Graph(id = 'line_daily') 
        ], width = 12)
    ]),
    
    #placeholder to build line graph for cumulative values
    dbc.Row([
        dbc.Col([
            dcc.Graph(id = 'line_cumulative')
        ], width = 12)
    ]),
    
    #create a radio selection for either 'Daily' or 'Cumulative' filter for map
    dbc.Row([
        dbc.Col([
             dcc.RadioItems(
                id = 'radio2',
                options=[
                    {'label': 'Daily', 'value': 'Daily'},
                    {'label': 'Cumulative', 'value': 'Cumulative'}
                ],
                value = 'Cumulative',
                labelStyle = {'padding': '5px', 'margin': 'auto'}
            )
        ])
    ]),
    
    dbc.Row([
        dbc.Col([
            html.P('Select Date:',
                   style = {'textDecoration': "underline"}),
            #create date selecter for the date of the data that is shown on the map
            dcc.DatePickerSingle(
                id='date_picker',
                min_date_allowed = cases['date'].dt.date.min(),
                max_date_allowed = cases['date'].dt.date.max(),
                initial_visible_month = today,
                date = today),
            #placeholder to build map
            dcc.Graph(id = 'map_cases')
        ], width = 12)
    ])
])

#first callback that uses the first radio selecter and county dropdown to build line graphs
@app.callback(
    Output('line_daily', 'figure'),
    Output('line_cumulative', 'figure'),
    Input('dropdown', 'value'),
    Input('radio', 'value')
)

def update_graph(input1, input2):  
    y_axis_daily = ''
    y_axis_cumulative = ''
    title_daily = ''
    title_cumulative = ''
    
    if (input2 == 'Cases'):
        y_axis_daily = 'cases'
        y_axis_cumulative = 'cumulative_cases'
        title_daily = 'Daily Number of Cases'
        title_cumulative = 'Cumulative Number of Cases'
    elif (input2 == 'Vaccine'):
        y_axis_daily = 'fully_vaccinated'
        y_axis_cumulative = 'cumulative_fully_vaccinated'
        title_daily = 'Daily Number of Fully Vaccinated'
        title_cumulative = 'Cumulative Number of Vaccinated'
    else:
        y_axis_daily = 'deaths'
        y_axis_cumulative = 'cumulative_deaths'
        title_daily = 'Daily Number of Deaths'
        title_cumulative = 'Cumulative Number of Deaths'
    
    filtered_table = filter_by_county(input1)
    
    fig = px.line(
        filtered_table,
        x = 'date',
        y = y_axis_daily,
        title = title_daily,
        color = 'area'
    )
    
    fig2 = px.line(
        filtered_table,
        x = 'date',
        y = y_axis_cumulative,
        title = title_cumulative,
        color = 'area'
    )
    return fig, fig2

#second callback that uses the date picker and radio selecters to build map
@app.callback(
    Output('map_cases', 'figure'),
    Input('date_picker', 'date'),
    Input('radio', 'value'),
    Input('radio2', 'value')
)

def update_map(date, value1, value2):
    color_input = ''
    
    if (value1 == 'Cases'):
        if (value2 == 'Daily'):
            color_input = 'cases_per_100k_pop'
        else:
            color_input = 'cumulative_cases_per_100k_pop'
    elif (value1 == 'Vaccine'):
        if (value2 == 'Daily'):
            color_input = 'fully_vaccinated_per_100k_pop'
        else:
            color_input = 'cumulative_vaccination_proportion'
    else:
        if (value2 == 'Daily'):
            color_input = 'deaths_per_100k_pop'
        else:
            color_input = 'cumulative_deaths_per_100k_pop'
    
    filtered_table = filter_by_date(date)
    filtered_table = filtered_table[filtered_table['area'] != 'California']
    
    fig = px.choropleth(
            filtered_table, 
            geojson=counties, 
            locations='area',
            featureidkey = 'properties.name', 
            color = color_input,
            color_continuous_scale = 'OrRd', 
            projection = 'mercator',
    )
    
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig

if __name__ == '__main__':
    app.run_server(debug = True, use_reloader = False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
